In [1]:
# This file has been modified from:
# https://github.com/DEAP/deap/blob/master/examples/ga/xkcd.py


#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.
"""This example shows a possible answer to a problem that can be found in this
xkcd comics: http://xkcd.com/287/. In the comic, the characters want to get
exactly 15.05$ worth of appetizers, as fast as possible."""

'This example shows a possible answer to a problem that can be found in this\nxkcd comics: http://xkcd.com/287/. In the comic, the characters want to get\nexactly 15.05$ worth of appetizers, as fast as possible.'

In [2]:
import random
from operator import attrgetter
from collections import Counter

# We delete the reduction function of the Counter because it doesn't copy added
# attributes. Because we create a class that inherits from the Counter, the
# fitness attribute was not copied by the deepcopy. This is just a DEAP technicality
# due to the fact we're using a standard Python class for our Individuals
try:
    del Counter.__reduce__
except: pass

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [3]:
# Create the item dictionary: item id is an integer, and value is 
# a (name, weight, value) 3-uple. Since the comic didn't specified a time for
# each menu item, we'll create our own times.
ITEMS = {'French Fries': (2.75, 5),
         'Hot Wings': (3.55, 7),
         'Mixed Fruit': (2.15, 2),
         'Mozzarella Sticks': (4.2, 4),
         'Sampler Plate': (5.8, 10),
         'Side Salad': (3.35, 3)}
ITEMS_NAME = list(ITEMS.keys())

In [4]:
# Our fitness function will have three values:
# cost difference from target price (we want to spend as much as possible but not more than our budget),
# time to eat (max of time to each individual appetizers),
# amount of food (sum of counts of appetizers)
#
# -1.0 means we are minimzing that value, 1.0 means we are maximizing
creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0, 1.0))

# Our "individual" will be a dictionary where the values are counts,
# so, a Python Counter object; we'll look at example individuals below.
creator.create("Individual", Counter, fitness=creator.FitnessMulti)

# initially, create individuals that have 2 items
IND_INIT_SIZE = 2

toolbox = base.Toolbox()
# individuals will be made up of counts associated with food names (from ITEM_NAMES)
toolbox.register("attr_item", random.choice, ITEMS_NAME)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [5]:
def evalXKCD(individual, target_price):
    """Evaluates the fitness and return the error on the price and the time
    taken by the order if the chef can cook everything in parallel."""
    price = 0.0
    times = [0]
    food = 0
    for item, number in individual.items():
        if number > 0:
            price += ITEMS[item][0] * number
            times.append(ITEMS[item][1])
            food += number
    return (price - target_price), max(times), food

In [6]:
# four french fries = $2.75 * 2 = $5.50, 2 side salads = $3.35 * 2 = $6.70,
# sum is $12.20, which is $2.85 under our target $15.05
#
# max time to eat all this is max time of the individual foods involved,
# which is 5... for french fries
#
# amount of food is 2+2 = 4
evalXKCD(creator.Individual({'French Fries': 2, 'Hot Wings': 0, 'Mixed Fruit': 0,
                             'Mozzarella Sticks': 0, 'Sampler Plate': 0, 'Side Salad': 2}),
         15.05)

(-2.8500000000000014, 5, 4)

In [7]:
def cxCounter(ind1, ind2, indpb):
    """Swaps the number of randomly-chosen items between two individuals"""
    for key in ITEMS.keys():
        if random.random() < indpb:
            ind1[key], ind2[key] = ind2[key], ind1[key]
    return ind1, ind2

In [8]:
cxCounter(creator.Individual({'French Fries': 4, 'Hot Wings': 0, 'Mixed Fruit': 0,
                              'Mozzarella Sticks': 0, 'Sampler Plate': 0, 'Side Salad': 3}),
          creator.Individual({'French Fries': 0, 'Hot Wings': 1, 'Mixed Fruit': 0,
                              'Mozzarella Sticks': 0, 'Sampler Plate': 1, 'Side Salad': 2}),
          0.50)

(Individual({'French Fries': 0,
             'Hot Wings': 0,
             'Mixed Fruit': 0,
             'Mozzarella Sticks': 0,
             'Sampler Plate': 0,
             'Side Salad': 2}),
 Individual({'French Fries': 4,
             'Hot Wings': 1,
             'Mixed Fruit': 0,
             'Mozzarella Sticks': 0,
             'Sampler Plate': 1,
             'Side Salad': 3}))

In [9]:
def mutCounter(individual):
    """Adds or remove an item from an individual"""
    if random.random() > 0.5:
        individual.update([random.choice(ITEMS_NAME)])
    else:
        val = random.choice(ITEMS_NAME)
        individual.subtract([val])
        if individual[val] < 0:
            del individual[val]
    return individual,

In [10]:
mutCounter(creator.Individual({'French Fries': 4, 'Hot Wings': 0, 'Mixed Fruit': 0,
                               'Mozzarella Sticks': 0, 'Sampler Plate': 0, 'Side Salad': 3}))

(Individual({'French Fries': 4,
             'Hot Wings': 0,
             'Mixed Fruit': 1,
             'Mozzarella Sticks': 0,
             'Sampler Plate': 0,
             'Side Salad': 3}),)

In [11]:
import sys

# register the functions, some with extra fixed arguments (like target_price)
toolbox.register("evaluate", evalXKCD, target_price=15.05)
# severely penalize individuals that are over budget ($15.05))
# return a fixed terrible fitness
toolbox.decorate("evaluate", tools.DeltaPenalty(lambda ind: evalXKCD(ind, 15.05)[0] <= 0,
                                               (-sys.float_info.max,
                                                sys.float_info.max,
                                                -sys.float_info.max)))
toolbox.register("mate", cxCounter, indpb=0.5)
toolbox.register("mutate", mutCounter)
toolbox.register("select", tools.selNSGA2)

In [12]:
# Simulation parameters:
# Number of generations = 100
NGEN = 100
# The number of individuals to select for the next generation (eliminate bad ones).
MU = 50
# The number of children to produce at each generation.
LAMBDA = 20
# The probability that an offspring is produced by crossover.
CXPB = 0.3
# The probability that an offspring is produced by mutation.
MUTPB = 0.3

# Start population at 50
pop = toolbox.population(n=MU)

In [13]:
# show initial pop
pop

[Individual({'Mixed Fruit': 1, 'Side Salad': 1}),
 Individual({'Mozzarella Sticks': 1, 'Side Salad': 1}),
 Individual({'Mixed Fruit': 1, 'Mozzarella Sticks': 1}),
 Individual({'French Fries': 1, 'Mixed Fruit': 1}),
 Individual({'French Fries': 2}),
 Individual({'Hot Wings': 1, 'Mixed Fruit': 1}),
 Individual({'French Fries': 1, 'Mixed Fruit': 1}),
 Individual({'French Fries': 1, 'Mozzarella Sticks': 1}),
 Individual({'Mozzarella Sticks': 2}),
 Individual({'Hot Wings': 1, 'Mozzarella Sticks': 1}),
 Individual({'Mozzarella Sticks': 1, 'Sampler Plate': 1}),
 Individual({'French Fries': 1, 'Sampler Plate': 1}),
 Individual({'Hot Wings': 1, 'Mozzarella Sticks': 1}),
 Individual({'Mozzarella Sticks': 1, 'Sampler Plate': 1}),
 Individual({'French Fries': 1, 'Sampler Plate': 1}),
 Individual({'Mozzarella Sticks': 1, 'Side Salad': 1}),
 Individual({'French Fries': 1, 'Sampler Plate': 1}),
 Individual({'Mozzarella Sticks': 1, 'Sampler Plate': 1}),
 Individual({'Hot Wings': 1, 'Side Salad': 1}),


In [14]:
# Also keep track of the "hall of fame" individual

# "The hall of fame contains the best individual that ever lived
# in the population during the evolution." DEAP docs

# Pareto Front means the best individual is the one that is equal
# or better on all dimensions of the fitness function; ours has
# two dimensions (cost difference from target and max time),
# so the HOF individual is the one that is lowest on both
hof = tools.ParetoFront()

# compute some statistics as the simulation proceeds
price_stats = tools.Statistics(key=lambda ind: ind.fitness.values[0])
time_stats = tools.Statistics(key=lambda ind: ind.fitness.values[1])
food_stats = tools.Statistics(key=lambda ind: ind.fitness.values[2])

stats = tools.MultiStatistics(price=price_stats, time=time_stats, food=food_stats)
stats.register("avg", numpy.mean, axis=0)


# run the simulation
algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN,
                          stats, halloffame=hof)

   	      	food	price 	time
   	      	--- 	------	----
gen	nevals	avg 	avg   	avg 
0  	50    	2   	-7.525	6.82
1  	14    	2.1 	-6.854	6.72
2  	9     	2.18	-6.477	6.8 
3  	14    	2.16	-5.926	7.46
4  	11    	2.36	-5.618	6.76
5  	9     	2.42	-6.011	5.72
6  	11    	2.5 	-6.146	5.02
7  	12    	2.5 	-6.108	5.08
8  	11    	2.6 	-5.962	4.98
9  	12    	2.68	-6.007	4.66
10 	17    	2.92	-5.012	5.02
11 	7     	2.94	-4.962	5.06
12 	13    	2.94	-5.225	4.74
13 	11    	2.78	-6.12 	4.22
14 	10    	2.78	-5.829	4.52
15 	12    	3.34	-3.51 	5.42
16 	12    	3.24	-3.973	5.54
17 	13    	3.14	-4.362	5.2 
18 	12    	3.38	-3.104	5.8 
19 	13    	3.44	-2.813	5.96
20 	14    	3.28	-3.382	5.74
21 	10    	3.5 	-2.653	6.18
22 	11    	3.42	-2.818	6.76
23 	11    	3.92	-0.906	7.82
24 	12    	3.92	-0.951	7.52
25 	17    	3.92	-0.933	7.64
26 	14    	3.92	-0.978	7.34
27 	15    	3.92	-1.034	6.98
28 	12    	3.92	-1.007	7.16
29 	13    	3.9 	-1.232	6.64
30 	12    	3.82	-1.532	6.44
31 	10    	3.82	-1.505	6.62
32 	9     	3.8 	-1.6

([Individual({'French Fries': 1,
              'Mixed Fruit': 1,
              'Mozzarella Sticks': 1,
              'Sampler Plate': 1,
              'Side Salad': 0}),
  Individual({'French Fries': 1,
              'Hot Wings': 0,
              'Mixed Fruit': 1,
              'Mozzarella Sticks': 1,
              'Sampler Plate': 0,
              'Side Salad': 0}),
  Individual({'French Fries': 1,
              'Hot Wings': 0,
              'Mixed Fruit': 1,
              'Mozzarella Sticks': 1,
              'Sampler Plate': 1,
              'Side Salad': 0}),
  Individual({'French Fries': 1,
              'Hot Wings': 0,
              'Mixed Fruit': 1,
              'Mozzarella Sticks': 1,
              'Sampler Plate': 1,
              'Side Salad': 0}),
  Individual({'French Fries': 1,
              'Hot Wings': 0,
              'Mixed Fruit': 1,
              'Mozzarella Sticks': 1,
              'Sampler Plate': 1,
              'Side Salad': 0}),
  Individual({'French Fries': 

In [15]:
for i in hof:
    print(i, i.fitness.values)

Individual({'Mixed Fruit': 1, 'Mozzarella Sticks': 1, 'French Fries': 1, 'Sampler Plate': 1, 'Hot Wings': 0}) (-0.15000000000000213, 10.0, 4.0)
Individual({'Mixed Fruit': 1, 'Mozzarella Sticks': 1, 'French Fries': 1, 'Sampler Plate': 1, 'Hot Wings': 0, 'Side Salad': 0}) (-0.15000000000000213, 10.0, 4.0)
Individual({'Mozzarella Sticks': 1, 'Mixed Fruit': 1, 'French Fries': 1, 'Sampler Plate': 1, 'Side Salad': 0}) (-0.15000000000000213, 10.0, 4.0)
Individual({'Mixed Fruit': 1, 'Mozzarella Sticks': 1, 'French Fries': 1, 'Sampler Plate': 1}) (-0.15000000000000213, 10.0, 4.0)
Individual({'Mixed Fruit': 3, 'Mozzarella Sticks': 2, 'Hot Wings': 0, 'Side Salad': 0}) (-0.20000000000000107, 4.0, 5.0)
Individual({'Mixed Fruit': 3, 'Mozzarella Sticks': 2, 'French Fries': 0, 'Hot Wings': 0, 'Side Salad': 0}) (-0.20000000000000107, 4.0, 5.0)
Individual({'Mixed Fruit': 3, 'Mozzarella Sticks': 2, 'Hot Wings': 0, 'Sampler Plate': 0, 'Side Salad': 0}) (-0.20000000000000107, 4.0, 5.0)
Individual({'Mixed F

In [16]:
print("Best:", hof[0], hof[0].fitness.values)

Best: Individual({'Mixed Fruit': 1, 'Mozzarella Sticks': 1, 'French Fries': 1, 'Sampler Plate': 1, 'Hot Wings': 0}) (-0.15000000000000213, 10.0, 4.0)
